In [152]:
import pandas as pd
data = pd.read_csv('D:/Users/HYUN/Desktop/News_Timeline/data/education_df.csv')

In [153]:
text = data.loc[0, 'body']

In [154]:
text

'한국교육과정평가원이 설립을 추진하고 있는 국가고사 출제센터(분리형) 예상 조감도 [사진 한국교육과정평가원]\n대학수학능력시험(수능)을 출제하는 한국교육과정평가원(평가원)이 국가고사를 출제하는 전담 시설 설립을 추진하고 있다. 지금까지 매년 비밀리에 숙박업소를 빌려 수능을 출제했지만, 보안과 비용 측면에서 비효율적이라는 이유다.\n \n31일 교육계에 따르면 평가원은 최근 \'국가고사 출제센터 설립 방안\' 보고서를 발표했다. 보고서는 수능과 초·중등 임용시험 등을 출제하는 시설인 \'국가고사 출제센터\'의 설립 필요성에 대한 내용을 담았다. 평가원은 빠르면 2024년에 출제센터를 설립한다는 계획이다.\n \n\'첩보 작전\' 같은 수능 출제\n수능 출제는 매년 첩보 작전을 방불케 하는 보안 속에 이뤄진다. 출제 경험이 있는 한 교수는 "출제위원이 됐다는 문자가 오면 주변 사람에게도 알리지 못하고, 평가원이 보낸 버스를 타고 이동한다"며 "출제 시설은 마치 공사 중인 건물처럼 바깥에 비계와 가림막을 설치해두기도 한다"고 말했다.\n \n평가원은 임대한 숙박업소의 위치와 상호도 밝히지 않는다. 관광객 등을 대상으로 한 업소를 빌리다 보니 시험을 치를 때마다 각종 보안 장비를 새로 설치한다. 야외활동을 하기 어려운 숙소에서 35일 동안 갇혀 있어야 하다 보니 출제위원 제안을 마다하는 경우도 많다.\n \n2018년 교육부 산하 기관이 주관한 9개 시험 출제에 들어간 임대료 등 비용 [표 한국교육과정평가원]\n시험 출제에 객실료만 88억원…"연간 35억원 절약 가능"\n보고서에 따르면 2018년 기준으로 수능 출제를 위한 객실료만 26억여원이다. 수능 모의평가 출제를 위한 객실료도 19억원이 들었다. 이 밖에 평가원이 출제하는 교사 임용시험이나 검정고시 등 9개 시험 출제에 쓴 객실료만 88억원에 달한다. 여기에 식음료비가 60억원 등 연간 출제에 쓴 임차료가 151억원에 이른다. 보고서는 출제 전담 시설을 지으면 한 해에 35억원을 아낄 수 있다고 추산했다.\n 

In [157]:
# 페이지랭크 계산에 networkx 패키지를 사용하므로, networkx 우선 설치
# ! pip install networkx

In [158]:
import networkx
import re

In [159]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

In [160]:

class RawSentenceReader:
    def __init__(self, textIter):
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

In [161]:
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0

    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
    
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

In [162]:
from konlpy.tag import Komoran

In [163]:
tagger = Komoran()

In [166]:
tr = TextRank()
print('Load...')
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
tr.loadSents(RawSentenceReader(text), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent))) # 'NNG'일반명사, 'NNP'고유명사, 'VV'동사, 'VA'형용사
print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    # 문장번호, 페이지랭크, 문장원문 순서로 출력
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))

Load...
Build...
4	0.055865510340723894	 평가원은 빠르면 2024년에 출제센터를 설립한다는 계획이다.
15	0.04977864896507011	평가원은 안정성과 보안도 센터 건립의 근거로 꼽았다.
16	0.04913090146928172	 현재 출제 시설로 쓰는 리조트나 유스호스텔 등 숙박업소의 보안이 취약하다는 것이다.
21	0.0490249601000795	평가원은 충북 청주시와 강원 평창군의 등을 출제센터 후보지로 꼽고 있다.
2	0.04880467793300228	31일 교육계에 따르면 평가원은 최근 '국가고사 출제센터 설립 방안' 보고서를 발표했다.
11	0.04737238094534879	 수능 모의평가 출제를 위한 객실료도 19억원이 들었다.
10	0.04678290887467502	보고서에 따르면 2018년 기준으로 수능 출제를 위한 객실료만 26억여원이다.
14	0.04655755275633939	 보고서는 출제 전담 시설을 지으면 한 해에 35억원을 아낄 수 있다고 추산했다.
29	0.045645185780831585	 평가원은 내년에도 센터 건립을 계속 추진할 방침이다.
5	0.045622498403870906	수능 출제는 매년 첩보 작전을 방불케 하는 보안 속에 이뤄진다.
0	0.04448024546229008	대학수학능력시험(수능)을 출제하는 한국교육과정평가원(평가원)이 국가고사를 출제하는 전담 시설 설립을 추진하고 있다.
12	0.041040021886365505	 이 밖에 평가원이 출제하는 교사 임용시험이나 검정고시 등 9개 시험 출제에 쓴 객실료만 88억원에 달한다.
3	0.040622607043385744	 보고서는 수능과 초·중등 임용시험 등을 출제하는 시설인 '국가고사 출제센터'의 설립 필요성에 대한 내용을 담았다.
23	0.03996613076639555	 센터 규모는 수능 출제 인원 700명과 지원 인력 300명이 함께 이용할 수 있는 1000명 규모로 예상된다.
1	0.03658380378

In [167]:
# 중요한 문장 순서로 정렬 후 문서의 20% 출력 
print(tr.summarize(0.2))

31일 교육계에 따르면 평가원은 최근 '국가고사 출제센터 설립 방안' 보고서를 발표했다.  평가원은 빠르면 2024년에 출제센터를 설립한다는 계획이다.  수능 모의평가 출제를 위한 객실료도 19억원이 들었다. 평가원은 안정성과 보안도 센터 건립의 근거로 꼽았다.  현재 출제 시설로 쓰는 리조트나 유스호스텔 등 숙박업소의 보안이 취약하다는 것이다. 평가원은 충북 청주시와 강원 평창군의 등을 출제센터 후보지로 꼽고 있다.
